In [1]:
%run ../Python_files/util.py
%run ../Python_files/util_ano_detec.py
%run ../Python_files/load_dicts.py

traffic_data_with_anomaly = zload('../temp_files/traffic_data_with_anomaly.pkz')
traffic_data_ref = zload('../temp_files/traffic_data_ref.pkz')

No dicts found; please check load_dicts...


/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def quantize(x, n, inf, sup):
    """
    x: the input data (scalar)
    n: quantization level
    inf: the lower bound of the input data x
    sup: the upper bound of the input data x
    -------------------
    Example:
    >>> quantize(4, 3, 2, 8)
    >>> 1
    >>> quantize(6.9, 3, 2, 8)
    >>> 2
    """
    if (x < inf):
        return 0
    elif (x >= sup):
        return (n-1)
    else:
        lev_length = (sup - inf) / float(n)
        return int(np.floor((x - inf) / lev_length))

In [3]:
tmc = '129-04138'
month = 1
day_list = [2, 3, 4, 5, 6]

traffic_data_ref_list = []
for hour in range(24):
    for minute in range(60):
        for day in day_list:
            key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
            traffic_data_ref_list.append(traffic_data_ref[key])

In [4]:
N = 10
inf = min(traffic_data_ref_list)
sup = max(traffic_data_ref_list)

In [5]:
traffic_data_ref_list_quantized = [quantize(s, N, inf, sup) for s in traffic_data_ref_list]

In [6]:
mapping_dict = {}

for i in range(N):
    for j in range(N):
        mapping_dict[(i, j)] = i * N + j

In [7]:
traffic_data_ref_list_quantized_trans = [mapping_dict[(traffic_data_ref_list_quantized[i], \
                                                       traffic_data_ref_list_quantized[i+1])] \
                                         for i in range(len(traffic_data_ref_list_quantized)-1)]

In [8]:
tmc = '129-04138'
month = 1
day = 9

traffic_data_with_anomaly_list = []
for hour in range(24):
    for minute in range(60):
        key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
        traffic_data_with_anomaly_list.append(traffic_data_with_anomaly[key])

In [9]:
traffic_data_with_anomaly_list_quantized = [quantize(s, N, inf, sup) for s in traffic_data_with_anomaly_list]

In [10]:
traffic_data_with_anomaly_list_quantized_trans = \
[mapping_dict[(traffic_data_with_anomaly_list_quantized[i], \
               traffic_data_with_anomaly_list_quantized[i+1])] \
 for i in range(len(traffic_data_with_anomaly_list_quantized)-1)]

In [11]:
mu_1 = mu_est(traffic_data_ref_list_quantized_trans, N)  # normal PL

In [12]:
mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

zdump([mu1, mu_11, P1, G_11, H_11, U_11], '../temp_files/Traffic_ano_detec_PLs.pkz')

<string>:443: RuntimeWarning: covariance is not positive-semidefinite.


In [13]:
num_of_samp_per_win = 50
num_test_sample = 24 * 60 - num_of_samp_per_win
beta = 0.001

In [14]:
eta_wc = {}
eta_Sanov = {}

n = num_of_samp_per_win

# Get thresholds for Hoeffding's test corresponding to sample length n    
key = str(n) + '_' + str(beta)
G_list = [G_11]
H_list = [H_11]
U_list = [U_11]
eta_1 = HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)
eta_2 =  - log(beta) / n
eta_wc[key] = eta_1
eta_Sanov[key] = eta_2
zdump([eta_wc, eta_Sanov], '../temp_files/testSample_threshold_traffic_ano_detec_%d.pkz'%N)

In [15]:
time_range = range(num_test_sample)

eta_wc_list = []
eta_Sanov_list = []
for idx in time_range:
    eta_wc_list.append(np.array(eta_wc[key]).tolist())
    eta_Sanov_list.append(np.array(eta_Sanov[key]).tolist())

In [16]:
test_sample = []

for idx in range(num_test_sample):
    test_sample.append(traffic_data_with_anomaly_list_quantized_trans[idx : (idx+60)])

In [17]:
KL = []
key = str(n) + '_' + str(beta)
for idx in range(num_test_sample):
    KL.append(KL_est(test_sample[idx], mu_11))
        
zdump(KL, '../temp_files/KL_%d_traffic_ano_detec.pkz'%N)

In [23]:
plot_points(time_range, KL, eta_wc_list)
plt.savefig('../temp_files/detec_results_N_%s_WC.eps'%N)

plt.ylabel('divergence')
plt.xlabel('time (min)')

plt.show()

In [22]:
plot_points(time_range, KL, eta_Sanov_list)
plt.savefig('../temp_files/detec_results_N_%s_Sanov.eps'%N)

plt.ylabel('divergence')
plt.xlabel('time (min)')

plt.show()